<a href="https://colab.research.google.com/github/gael-roustan-epsi-2022/ia-notebooks/blob/main/Linear_Regression_with_a_Real_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Copyright 2020 Google LLC. Double-click here for license information.
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Linear Regression with a Real Dataset

This Colab uses a real dataset to predict the prices of houses in California.   






## Objectifs de cet exercice :

A la fin de cet exercice, vous serez en mesure de :
  * lire un fichier .csv enregistré dans un objet [pandas](https://developers.google.com/machine-learning/glossary/#pandas) DataFrame.
  * Analyser un [jeu de données](https://developers.google.com/machine-learning/glossary/#data_set). 
  * Expérimenter avec plusieurs [caractéristiques](https://developers.google.com/machine-learning/glossary/#feature) pour construire un modèle.
  * Affiner le modèle et notamment ses [hyperparamètres](https://developers.google.com/machine-learning/glossary/#hyperparameter).

## Le jeu de données

Cet exercice est à réaliser avec 2 jeux de données.

Le 1er [jeu de données](https://developers.google.com/machine-learning/crash-course/california-housing-data-description) est basé sur les données immobilières de Californie.

Le 2nd [jeu de données]() est basé sur les données immobilières de France.
Attention, les étapes décrites pour nettoyer la donnée et le nom des colonnes ne correspond que pour le 1er jeu de données. A vous d'adapter pour le 2nd jeu de données.

## Utiliser la bonne version de tensorflow

Le code caché s'assure que Colab utilise la bonne version de Tensorflow, à savoir 2.X


In [ ]:
#@title Run on TensorFlow 2.x
%tensorflow_version 2.x

## Imporer les modules les plus importants

Le code suivant importe les modules nécessaires au reste du code dans colab.


In [ ]:
#@title Import relevant modules
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt

# The following lines adjust the granularity of reporting. 
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

## Le jeu de données

Les jeux de données sont souvent stockés sur disque ou disponible à une adresse internet au [format csv](https://wikipedia.org/wiki/Comma-separated_values). 

Un fichier csv correctement formaté contient les noms des colonnes dans la première ligne, suivi par plusieurs lignes de données. Une virgule sépare chaque valeur sur chaque ligne. Par exemple, ci-dessous les cinq premières lignes du fichier csv contenant les données immobilières de Californie.

```
"longitude","latitude","housing_median_age","total_rooms","total_bedrooms","population","households","median_income","median_house_value"
-114.310000,34.190000,15.000000,5612.000000,1283.000000,1015.000000,472.000000,1.493600,66900.000000
-114.470000,34.400000,19.000000,7650.000000,1901.000000,1129.000000,463.000000,1.820000,80100.000000
-114.560000,33.690000,17.000000,720.000000,174.000000,333.000000,117.000000,1.650900,85700.000000
-114.570000,33.640000,14.000000,1501.000000,337.000000,515.000000,226.000000,3.191700,73400.000000
```

Pour les valeurs foncières de France, nous avons plus de colonnes, donc sûrement plus d'étapes de préparation de données à venir
```
id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,code_commune,nom_commune,code_departement,ancien_code_commune,ancien_nom_commune,id_parcelle,ancien_id_parcelle,numero_volume,lot1_numero,lot1_surface_carrez,lot2_numero,lot2_surface_carrez,lot3_numero,lot3_surface_carrez,lot4_numero,lot4_surface_carrez,lot5_numero,lot5_surface_carrez,nombre_lots,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude
2021-1,2021-01-05,000001,Vente,185000,5080,,CHE DE VOGELAS,0471,01370,01426,Val-Revermont,01,,,01426312ZC0122,,,,,,,,,,,,,0,1,Maison,97,5,S,sols,,,2410,5.386094,46.32714
2021-1,2021-01-05,000001,Vente,185000,5080,,CHE DE VOGELAS,0471,01370,01426,Val-Revermont,01,,,01426312ZC0122,,,,,,,,,,,,,0,3,DÃ©pendance,,0,S,sols,,,2410,5.386094,46.32714
2021-2,2021-01-06,000001,Vente,10,,,ROUGEMONT,B043,01290,01042,Bey,01,,,010420000A0204,,,,,,,,,,,,,0,,,,,BT,taillis simples,,,530,4.844368,46.224233
2021-3,2021-01-04,000001,Vente,204332,7,,ALL DES ECUREUILS,0276,01310,01065,Buellas,01,,,010650000B1325,,,,,,,,,,,,,0,1,Maison,88,4,S,sols,,,866,5.157688,46.200988
```


### Charger le fichier csv dans un objet DataFrame Pandas

Ce fichier Colab, comme beaucoup d'autres programmes de Machine Learning, rassemble les fichiers csv et les stocke en mémoire en tant qu'objet DataFrame Pandas.
Pandas est une bibliothèque open source Python.
Le type de structure de données primaire dans Pandas est le DataFrame.
Voyez le DataFrame comme une feuille de calcul dans laquelle chaque ligne est identifiée par une ligne et chaque colonne par un nom. Pandas s'appuie lui-même sur une bibliothèque Python open source appelé NumPy. Si vous n'êtes pas familier avec toutes ces technologies, vous pouvez consulter les liens suivants ou bien refaire les exercices précédents du cours !

*   [NumPy](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/numpy_ultraquick_tutorial.ipynb?utm_source=linearregressionreal-colab&utm_medium=colab&utm_campaign=colab-external&utm_content=numpy_tf2-colab&hl=en)
*   [Pandas DataFrames](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/pandas_dataframe_ultraquick_tutorial.ipynb?utm_source=linearregressionreal-colab&utm_medium=colab&utm_campaign=colab-external&utm_content=pandas_tf2-colab&hl=en)

Le code suivant importe le fichier csv dans un DataFrame Pandas et remet à l'échelle les valeurs contenues dans le label `median_house_value`.


In [ ]:
# Import the dataset.
training_df = pd.read_csv(filepath_or_buffer="https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv")

# Scale the label.
training_df["median_house_value"] /= 1000.0

# Print the first rows of the pandas DataFrame.
training_df.head()

Mettre à l'échelle `median_house_value` permet de changer l'unité de la valeur de chaque au millier. Le fait de mettre à l'échelle va nous permettre de conserver des valeurs d'écart (loss) et de taux d'apprentissage (learning rates) dans des intervalles plus pratiques.

Bien que mettre à l'échelle le label n'est *pas* essentielle, la mise à l'échelle des caractéristiques (features) dans un modèle à multi-caractéristique *est* essentielle.


## Analyser le dataset

Une grande partie des projets de machine learning consiste à bien connaître vos données.
L'API Pandas fournit une fonction `describe` qui affiche les statistiques suivantes pour chaque colonne dans l'objet DataFrame :

* `count`, qui est le nombre de lignes dans la colonne. Normalement, `count` renvoie la même valeur pour chaque colonne. 

* `mean` et `std`, qui correspondent à la moyenne et l'écart type des valeurs de chaque colonne. 

* `min` et `max`, qui correspondent à la plus petite et plus grande valeur de chaque colonne.

* `25%`, `50%`, `75%`, qui correspondent aux différents [quartiles](https://developers.google.com/machine-learning/glossary/#quantile).

In [ ]:
# Get statistics on the dataset.
training_df.describe()


### Exercice 1: Identifier les anomalies dans le jeu de données

Voyez-vous des anomalies (valeurs étranges) dans les données ?
Pensez à observer la répartition des données et à la définition des quartiles.

## Définir les fonctions de création et d'entraînement de modèle

Le code suivant définit 2 fonctions :

  * `build_model(my_learning_rate)`, qui construit un modèle initialisé avec des valeurs aléatoires.
  * `train_model(model, feature, label, epochs)`, qui entraîne le modèle à partir des données exemples (caractéristiques et labels) que vous lui passez. 

Pas la peine de modifier le code.


In [ ]:
#@title Define the functions that build and train a model
def build_model(my_learning_rate):
  """Create and compile a simple linear regression model."""
  # Most simple tf.keras models are sequential.
  model = tf.keras.models.Sequential()

  # Describe the topography of the model.
  # The topography of a simple linear regression model
  # is a single node in a single layer.
  model.add(tf.keras.layers.Dense(units=1, 
                                  input_shape=(1,)))

  # Compile the model topography into code that TensorFlow can efficiently
  # execute. Configure training to minimize the model's mean squared error. 
  model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=my_learning_rate),
                loss="mean_squared_error",
                metrics=[tf.keras.metrics.RootMeanSquaredError()])

  return model        


def train_model(model, df, feature, label, epochs, batch_size):
  """Train the model by feeding it data."""

  # Feed the model the feature and the label.
  # The model will train for the specified number of epochs. 
  history = model.fit(x=df[feature],
                      y=df[label],
                      batch_size=batch_size,
                      epochs=epochs)

  # Gather the trained model's weight and bias.
  trained_weight = model.get_weights()[0]
  trained_bias = model.get_weights()[1]

  # The list of epochs is stored separately from the rest of history.
  epochs = history.epoch
  
  # Isolate the error for each epoch.
  hist = pd.DataFrame(history.history)

  # To track the progression of training, we're going to take a snapshot
  # of the model's root mean squared error at each epoch. 
  rmse = hist["root_mean_squared_error"]

  return trained_weight, trained_bias, epochs, rmse

print("Defined the create_model and traing_model functions.")

## Définir les fonctions de visualisation

La fonction [matplotlib](https://developers.google.com/machine-learning/glossary/#matplotlib) ci-après crée les graphiques suivants :

*  a graphique de points qui représente la caractéristique en fonction du label, et une ligne qui montre la sortie du modèle entraîné
*  la courbe de l'écart calculé

Pas la peine de modifier le code ci-après.


In [ ]:
#@title Define the plotting functions
def plot_the_model(trained_weight, trained_bias, feature, label):
  """Plot the trained model against 200 random training examples."""

  # Label the axes.
  plt.xlabel(feature)
  plt.ylabel(label)

  # Create a scatter plot from 200 random points of the dataset.
  random_examples = training_df.sample(n=200)
  plt.scatter(random_examples[feature], random_examples[label])

  # Create a red line representing the model. The red line starts
  # at coordinates (x0, y0) and ends at coordinates (x1, y1).
  x0 = 0
  y0 = trained_bias
  x1 = 10000
  y1 = trained_bias + (trained_weight * x1)
  plt.plot([x0, x1], [y0, y1], c='r')

  # Render the scatter plot and the red line.
  plt.show()


def plot_the_loss_curve(epochs, rmse):
  """Plot a curve of loss vs. epoch."""

  plt.figure()
  plt.xlabel("Epoch")
  plt.ylabel("Root Mean Squared Error")

  plt.plot(epochs, rmse, label="Loss")
  plt.legend()
  plt.ylim([rmse.min()*0.97, rmse.max()])
  plt.show()  

print("Defined the plot_the_model and plot_the_loss_curve functions.")

## Appeler les fonctions du modèle

Une part importante du machine learning est de déterminer quelles [features (caractéristiques)](https://developers.google.com/machine-learning/glossary/#feature) sont corrélées au [label](https://developers.google.com/machine-learning/glossary/#label). Par exemple, dans la vie réelle, les modèles de prédiction de valeurs foncières s'appuient sur des centaines de caractéristiques. En revanche, notre modèle va s'appuyer sur une unique caractéristique. Pour l'instant, vous prenez de manière arbitraire la caractéristique `total_rooms`.


In [ ]:
# The following variables are the hyperparameters.
learning_rate = 0.01
epochs = 30
batch_size = 30

# Specify the feature and the label.
my_feature = "total_rooms"  # the total number of rooms on a specific city block.
my_label="median_house_value" # the median value of a house on a specific city block.
# That is, you're going to create a model that predicts house value based 
# solely on total_rooms.  

# Discard any pre-existing version of the model.
my_model = None

# Invoke the functions.
my_model = build_model(learning_rate)
weight, bias, epochs, rmse = train_model(my_model, training_df, 
                                         my_feature, my_label,
                                         epochs, batch_size)

print("\nThe learned weight for your model is %.4f" % weight)
print("The learned bias for your model is %.4f\n" % bias )

plot_the_model(weight, bias, my_feature, my_label)
plot_the_loss_curve(epochs, rmse)

Une grande part d'aléatoire intervient dans l'entraînement d'un modèle. Par conséquent, vous obtiendrez des résultats différents à chaque fois que vous entraînez un modèle identique.


## Utiliser le modèle entraîné pour réaliser des prédictions

Vous pouvez utiliser le modèle entraîné pour réaliser des prédictions. En pratique, [vous devriez faire des prédictions sur des données qui n'ont pas servi à l'entraînement](https://developers.google.com/machine-learning/crash-course/training-and-test-sets/splitting-data). Cepedant, pour cet exercice, vous allez travailler avec un sous ensemble du jeu de données d'entraînement. Plus loin, nous aborderons ce point.

Pour commener, exécuter le code suivant pour créer une fonction qui va réaliser une prédiction.


In [ ]:
def predict_house_values(n, feature, label):
  """Predict house values based on a feature."""

  batch = training_df[feature][10000:10000 + n]
  predicted_values = my_model.predict_on_batch(x=batch)

  print("feature   label          predicted")
  print("  value   value          value")
  print("          in thousand$   in thousand$")
  print("--------------------------------------")
  for i in range(n):
    print ("%5.0f %6.0f %15.0f" % (training_df[feature][10000 + i],
                                   training_df[label][10000 + i],
                                   predicted_values[i][0] ))

Maintenant, appeler cette fonction sur 10 exemples.

In [ ]:
predict_house_values(?, ?, ?)

### Exercice 2: Evaluer la qualité de la prédiction du modèle

Jeter un oeil à la table précédente. Est-ce que les données prédites sont proches des données attendues ?


## Exercice 3: Essayer une autre caractéristique

La caractéristique `total_rooms` avait un faible pouvoir de prédiction. Essayer maintenant avec la caractéristique `population`.

Note: Si vous changez les caractéristiques, il est fort probable que vous changerez les hyperparamètres.

In [ ]:
my_feature = ?   # Replace the ? with population or possibly
                   # a different column name.

# Experiment with the hyperparameters.
learning_rate = ?
epochs = ?
batch_size = ?

# Don't change anything below this line.
my_model = build_model(learning_rate)
weight, bias, epochs, rmse = train_model(my_model, training_df, 
                                         my_feature, my_label,
                                         epochs, batch_size)
plot_the_model(weight, bias, my_feature, my_label)
plot_the_loss_curve(epochs, rmse)

predict_house_values(15, my_feature, my_label)

Est-ce que la colonne `population` produit de meilleurs résultats que `total_rooms`?

## Exercice 4: Créer une caractéristique supplémentaire

Vous avez constaté que les caractéristiques `total_rooms` et `population` n'était pas pertinentes. Cela ne sera proablement pas le cas non plus pour les autres colonnes.
Peut être que le *ratio* de `total_rooms` par `population` pourrait avoir un pouvoir de prédiction plus important. Et peut être que la densité à un rapport avec la valeur immobilière.

Pour tester ces hypothèses, réaliser les actions suivantes :

1. Créer une [caractéristique supplémentaire](https://developers.google.com/machine-learning/glossary/#synthetic_feature) qui est le ratio entre `total_rooms` et `population`. (Si vous êtes nouveau avec les dataframes Pandas, merci de suivre le tutorial suivant [Pandas DataFrame Ultraquick Tutorial](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/pandas_dataframe_ultraquick_tutorial.ipynb?utm_source=linearregressionreal-colab&utm_medium=colab&utm_campaign=colab-external&utm_content=pandas_tf2-colab&hl=en).)
2. Affiner les 3 hyperparamètres
3. Déterminer si la nouvelle caractéristiques supplémentaire permet de réduire l'écart par rapport à la simple caractéristique que vous avez essayé auparavant.

In [ ]:
# Define a synthetic feature named rooms_per_person
training_df["rooms_per_person"] = ? # write your code here.

# Don't change the next line.
my_feature = "rooms_per_person"

# Assign values to these three hyperparameters.
learning_rate = ?
epochs = ?
batch_size = ?

# Don't change anything below this line.
my_model = build_model(learning_rate)
weight, bias, epochs, rmse = train_model(my_model, training_df,
                                         my_feature, my_label,
                                         epochs, batch_size)

plot_the_loss_curve(epochs, rmse)
predict_house_values(15, my_feature, my_label)

En regardant la coubre de l'écart, cette caractéristique supplémentaire produit un bien meilleur modèle. Cependant, le modèle fait toujours des prédictions à côté de la plaque.


## Exercice 5. Trouver la caractéristique dont les valeurs brutes sont les mieux corrélées avec le label résultat

Maintenant que nous nous somes appuyés sur le tatonnement pour identifier les potentielles caractéristiques liées au label, essayons de travailler les statistiques.

Une **matrice de corrélation** indique comment chaque valeur brute d'attribute est lié aux autres valeurs brutes d'attribut. Les valeurs de corrélation ont la signification suivante :

  * `1.0`: corrélation positive parfaite. Quand cet attribut augmente, l'autre attribut augmente également..
  * `-1.0`: corrélation négative parfait, quand un attribut augmente, l'autre diminue. 
  * `0.0`: pas de corrélation; les 2 colonnes [ne sont pas corrélées](https://en.wikipedia.org/wiki/Correlation_and_dependence#/media/File:Correlation_examples2.svg).

En général, plus la valeur absolue de la valeur de corréaltion est importante, plus le pouvoir de prédiction est grand. Par exemple, une valeur de corrélation de -0.8 implique un pouvoir de prédiction plus important qu'une valeur de corréaltion de 0.2
Ci-après le code pour générer la matrice de corrélation :


In [ ]:
# Generate a correlation matrix.
training_df.corr()

Quelle est la caractéristique la plus appropriée ?

Les matrices de corrélation ne sont pas l'unique outil mais sont un excellent moyen pour démarrer.
